# RoboHive Environment Registration & Customization

## Register
Follow Gym registration API to register a RoboHive Environment

In [1]:
import gym
from gym.envs.registration import register
import os

In [2]:
# Hand Manipulation Suite: Open door
from mj_envs.envs.hand_manipulation_suite.door_v1 import DoorEnvV1
register(
    id='DemoDoor-v1',
    entry_point='mj_envs.envs.hand_manipulation_suite:DoorEnvV1',
    max_episode_steps=100,
    kwargs={
        'model_path':'../envs/hand_manipulation_suite/assets/DAPG_door.xml',
    }
)

RS:> Registering Arms Envs
RS:> Registering Myo Envs
RS:> Registering Hand Envs
RS:> Registering Appliances Envs
RS:> Registering Kitchen Envs
RS:> Registering Multi-Task (2 subtasks) Envs
RS:> Registering Multi-Task (9 subtasks) Envs
RS:> Registering Claw Envs


Lets test the newly registered environment

In [3]:
env = gym.make('DemoDoor-v1')
env.reset()
print(f"Time={env.time:.2f}", end=", ")
for _ in range(10):
    env.step(env.action_space.sample()) # take a random action
    print(f"{env.time:.2f}", end=", ")
env.close()

Time=0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 

## Configure
RoboHive aggressively uses `kwargs` to configure environments. There are three different ways to configure environments in RoboHive 

### 1. Passing `kwargs` during registration
In the examples below `kwargs` are used to specify which `robot_site_name` to use for reaching to which `target_site_name`. Additionally the randomization range of the target is specified as `target_xyz_range`.

In [4]:
# Franka arm to reach random targets
register(
    id='DemoFrankaReachRandom-v0',
    entry_point='mj_envs.envs.arms.reach_base_v0:ReachBaseV0',
    max_episode_steps=50, #50steps*40Skip*2ms = 4s
    kwargs={
        'model_path': '../envs/arms/franka/assets/franka_reach_v0.xml',
        'config_path': '../envs/arms/franka/assets/franka_reach_v0.config',
        'robot_site_name': "end_effector",
        'target_site_name': "target",
        'target_xyz_range': {'high':[0.3, .5, 1.2], 
                             'low':[-.3, .1, .8]
                            }
    }
)

Lets inspect one of the passed configurations

In [5]:
env = gym.make('DemoFrankaReachRandom-v0')
print("Target randomization range::\n", env.target_xyz_range)

Target randomization range::
 {'high': [0.3, 0.5, 1.2], 'low': [-0.3, 0.1, 0.8]}


### 2. Registering env variants
While every RoboHive env are packaged with care, research projects often require flexibility to customize the prepackaed environments. RoboHive provides functionality to easily create variants of preregistered environments. We have found this functionality to be really useful when multiple closely related envs are required. For example -- env variations during hyper parameter sweeps, testing a policy on diffeernt env conditions, system identification.

In [6]:
from mj_envs.envs.env_variants import register_env_variant
base_env_name = "kitchen-v3"

# Register a variant of the kitchen env
base_env_variants={
    'max_episode_steps':50,                     # special key
    'obj_goal': {"lightswitch_joint": -0.7},    # obj_goal keys will be updated
    'obs_keys_wt': {                            # obs_keys_wt will be updated
        'robot_jnt': 5.0,
        'obj_goal': 5.0,
        'objs_jnt': 5.0,}
}
variant_env_name = register_env_variant(env_id=base_env_name, variants=base_env_variants)

Registered a new env-variant: kitchen-hor_50-obj_goal.lightswitch_joint_-0.7-obs_keys_wt.robot_jnt_5.0-obs_keys_wt.obj_goal_5.0-obs_keys_wt.objs_jnt_5.0-v3


Lets inspect the two env variants. Pay attention to two details -- name and the updated specs. 
1. Note that env-variant picks up a new unique name to distinguish itself from the original env. Unique name can also be expliciely provided using `variant_id`
2. Variant env picks up the missing details from the base env. Information is merged by defaults. `override_keys` can be used to fully override any keys

In [7]:
print(f"Base-env kwargs: {base_env_name}")
print(gym.envs.registry.env_specs[base_env_name]._kwargs)
print(f"\nEnv-variant kwargs: {variant_env_name}")
print(gym.envs.registry.env_specs[variant_env_name]._kwargs)

Base-env kwargs: kitchen-v3
{'model_path': '/Users/vikashplus/Libraries/mj_envs/mj_envs/envs/multi_task/substeps1/../common/kitchen/franka_kitchen.xml', 'config_path': '/Users/vikashplus/Libraries/mj_envs/mj_envs/envs/multi_task/substeps1/../common/kitchen/franka_kitchen.config', 'obj_goal': {}, 'obj_init': {'knob1_joint': 0, 'knob2_joint': 0, 'knob3_joint': 0, 'knob4_joint': 0, 'lightswitch_joint': 0, 'slidedoor_joint': 0, 'micro0joint': 0, 'rightdoorhinge': 0, 'leftdoorhinge': 0}, 'obs_keys_wt': {'robot_jnt': 1.0, 'objs_jnt': 1.0, 'obj_goal': 1.0, 'end_effector': 1.0, 'knob1_site_err': 1.0, 'knob2_site_err': 1.0, 'knob3_site_err': 1.0, 'knob4_site_err': 1.0, 'light_site_err': 1.0, 'slide_site_err': 1.0, 'leftdoor_site_err': 1.0, 'rightdoor_site_err': 1.0, 'microhandle_site_err': 1.0, 'kettle_site0_err': 1.0}}

Env-variant kwargs: kitchen-hor_50-obj_goal.lightswitch_joint_-0.7-obs_keys_wt.robot_jnt_5.0-obs_keys_wt.obj_goal_5.0-obs_keys_wt.objs_jnt_5.0-v3
{'model_path': '/Users/vikashp

### 3. Passing `kwargs` during make (not advised)
In the examples below `kwargs` are used to change/boost the prespecified randomization range of the target

In [8]:
env_kwargs = {'target_xyz_range' : 
                 {'high':[0.4, .6, 1.3], 
                  'low':[-.2, .0, .7]
                 }
             }
env_new_range = gym.make('DemoFrankaReachRandom-v0', **(env_kwargs))

Lets inspect the passed configurations between the two envs

In [9]:
print(f"Original env: {env.id}\n target randomization range::\n", env.target_xyz_range)
print(f"Updated env: {env.id}\n target randomization range::\n", env_new_range.target_xyz_range)

Original env: DemoFrankaReachRandom-v0
 target randomization range::
 {'high': [0.3, 0.5, 1.2], 'low': [-0.3, 0.1, 0.8]}
Updated env: DemoFrankaReachRandom-v0
 target randomization range::
 {'high': [0.4, 0.6, 1.3], 'low': [-0.2, 0.0, 0.7]}


**NOTE:** Passing kwargs during make is not advisable as the updated env have the same id as the original env. This leads to two potential issues 

1. *Confusion while reporting results* - Reporting results using the env's name while its configurations have been changed leads to reproducibility issues. If env's configuration changes are needed, it is recommended to instead to use env_variant to register the altered name with its own unique id. This is a very common mistake in the field. Lets fix this! 
**Recommendation**: For reporting results RoboHive recommends projects/papers to use `register_env_variant` at top of their scripts to create a unique env name `<PAPERNAME>_FrankaReachRandom-v0` if default envs are customized in any way.

2. *Confusion while usage* - two env with the same id but different properties can lead to confusion during development/usage. 